In [1]:
path = 'REC NOT CLOSED VFinale2.xlsm'
import pandas as pd

In [2]:
def preprocess_data(path = '../RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed


    return wb

In [3]:
def preprocess_data(path = '../RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)


    return wb
        

In [4]:
wb = preprocess_data()

In [5]:
import plotly.express as px

In [6]:
import numpy as np

# Budgetted time vs time spent

In [7]:
from docx import Document
import pandas as pd

In [8]:
from os import listdir
from os.path import join as osjoin

In [9]:
file = '../OneDrive_1_05-09-2022/IRA Harare 24 June 2022.docx'


In [10]:
def extract_load(path = '../OneDrive_1_05-09-2022'):
    header_of_interest = ['Activity/ Persons-days','Planning','Fieldwork','Draft report','Finalization','Total']
    int_cols = ['Planning', 'Fieldwork', 'Draft report', 'Finalization', 'Total']
    dtypes = ['int64']*len(int_cols)

    doc_files = [i for i in listdir(path) if '.docx' in i]

    for f in doc_files:

        sheet_name = f[:-5]
   
        file = osjoin(path,f)
        print(sheet_name)
        doc = Document(file)
        tables = doc.tables

        
        for t in tables:
            header = [cell.text for cell in t.rows[0].cells]
            if header == header_of_interest:
                data = [[cell.text for cell in row.cells] for row in t.rows[1:]]
                break

        df = pd.DataFrame(data,columns = header_of_interest)

        for a in int_cols:
            df[a] = pd.to_numeric(df[a],errors = 'coerce',downcast='integer')    
        
        with pd.ExcelWriter('Audit budget days.xlsx',mode = 'a',engine='openpyxl',if_sheet_exists='replace') as writer:
            df.to_excel(writer,sheet_name=sheet_name,index=False)

        

In [11]:
# extract_load()

In [12]:
d = pd.read_excel('../Field Office Leadhseets.xlsx',index_col=False,sheet_name=None)

In [13]:
d.keys()

dict_keys(['IRA_Addis_20220616_after CSA', 'IRA IICBA', 'IRA Harare 24 June 2022', 'IRA Havana Final', 'IRA Yaounde', 'San José_Leadsheet ', 'IRA Teheran_Jan 2022'])

In [14]:
sheet_names = ['IRA_Addis_20220616_after CSA','IRA IICBA', 'IRA Harare 24 June 2022', 'IRA Havana Final', 'IRA Yaounde',
            'San José_Leadsheet ', 'IRA Teheran_Jan 2022']

In [15]:
offices = ['Addis','IICBA','Harare','Havana','Yaound','San Jos','Tehran']

In [16]:
connector = dict(zip(offices,sheet_names))

In [17]:
actual = wb[wb['Assignment Name'].str.contains('|'.join(offices),na=False,case=False)]

In [18]:
actual = actual.assign(Date=pd.to_datetime(actual['Date']))

In [19]:
actual['year'] = actual['Date'].dt.strftime('%Y')

In [20]:
actual['working_hours'] = actual['Project Hours'] + actual['Admin Hours']

In [21]:
for k in connector:
    key = connector[k]
    d[key]['name'] = d[key]['Activity/ Persons-days'].str.extract(r'(\S+$)')[0]
    d[key]['name'] = d[key]['name'].str.slice(1,-1)
    d[key]['Planning'] *= 7.5
    d[key]['Fieldwork'] *= 7.5
    d[key]['Draft report'] *= 7.5
    d[key]['Finalization'] *= 7.5
    d[key]['Total'] *= 7.5
    d[key]['office'] = k

In [22]:
p = ['Planning','Fieldwork','Draft report','Finalization']
a = ['1-Planning', '2-Fieldwork', '3-Reporting', '4-Finalization of Workpapers']

In [23]:
phase_mapper = dict(zip(a,p))

In [24]:
actual['surname'] = actual['Resource'].str.extract(r'(\s)(\w+)')[1]

In [25]:
budget = pd.DataFrame()

In [26]:
for b in d:
    budget = pd.concat((budget,d[b]))

In [27]:
budget = budget[budget['name']!='ota']#.drop(columns=['Total'])

In [28]:
ofs = ['('+i+')' for i in connector.keys()]
sd = actual['Assignment Name'].str.extract('|'.join(ofs))

In [29]:
ofs = pd.Series(dtype = 'string')

for c in sd.columns:
    ofs = pd.concat((ofs,sd[c][~sd[c].isna()]))

In [34]:
actual['office'] = ofs

In [35]:
phase_mapper = dict(zip(actual['Phase'].sort_values().unique(),budget.columns[1:]))
categories = ['office','phase']


In [36]:
actual['phase'] = ''
for k in phase_mapper:
    actual['phase'].mask(actual['Phase']==k,phase_mapper[k],inplace=True)

In [37]:
budget = budget.groupby(['office'],as_index=False).sum().melt(id_vars=['office'],var_name='phase',value_name='planned_hours',
value_vars=['Planning','Fieldwork','Draft report','Finalization','Total'])

In [38]:
budget['phase'].mask(budget['phase']=='Total','Overall',inplace=True)

In [39]:
totals = actual[actual.columns[-4:]].groupby(['office'],as_index=False).sum()

In [40]:
totals['phase']='Overall'

In [41]:
act = actual[actual.columns[-4:]].groupby(categories,as_index=False).sum()
act = pd.concat((act,totals))

In [42]:
budget = budget.groupby(categories).sum()

In [43]:
act.set_index(categories,inplace=True)

In [47]:
actual['Assignment Code'].unique()

array(['2022-FO-HAV', '2022-FO-San Jose', '2022-FO-Harare', '2021-FO-TEH',
       '2022-FO-Addis Ababa', '2022-FO-Yaounde', '2022-IN-IICBA'],
      dtype=object)

In [44]:
df = budget.join(act).reset_index()

In [40]:
df['diff'] = df['planned_hours']-df['working_hours']

In [41]:
df['status'] = 'Within budget'
df['status'].where(df['diff']>0,'Out of Budget',inplace=True)

In [42]:
df['city'] = df['office']
df['city'].mask(df['city'] == 'San Jos','San José', inplace=True)
df['city'].mask(df['city'] == 'Yaound','Yaounde', inplace=True)

In [43]:
df['phase_num'] = 1
for i in phase_mapper:
    df['phase_num'].mask(df['phase'] == phase_mapper[i],int(i[0]),inplace=True)

df['phase_num'].mask(df['phase'] == 'Overall',len(phase_mapper)+1,inplace=True)

In [44]:
(px.bar(df,y='phase',x='diff',facet_col='city',color='status',labels={'diff':"Budget - spent (hours)"},
    title='Difference between budgeted time and time spent per field office',
    category_orders={'phase':['Planning','Fieldwork','Draft report','Finalization','Overall']})
.update_xaxes(showgrid=False)
.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
.update_layout(title_x=0.5)
.for_each_xaxis(lambda xaxis: xaxis.update(title=''))
.add_annotation(y=-0.3,x=0.5,showarrow=False,text="Budget - actual (hours)", textangle=0,font_size=34,xref="paper", yref="paper")
)

In [45]:
e = {'planned_hours':'Budgeted hours','working_hours':'Spent hours'}

In [46]:
df['percentage'] = (100*df['working_hours']/df['planned_hours']).round(1)

In [47]:
df['disp'] = ''
df['disp'].mask(df['percentage']>=100,'+'+(df['percentage']-100).round(1).astype('string')+'%',inplace=True)
df['disp'].mask(df['percentage']<100,'-'+(100-df['percentage']).round(1).astype('string')+'%',inplace=True)
df['disp'].mask(df['percentage'].isna(),'-'+str(100)+'%',inplace=True)

In [48]:
df['colour'] = 'red'
df['colour'].mask((df['percentage']<=100)|(df['percentage'].isna()),'green',inplace=True)

In [49]:
cities = df['city'].sort_values().unique()
city_mapper = dict(zip(cities,range(1,len(cities)+1)))

In [50]:
cols = ['city','planned_hours','working_hours','diff','percentage','disp','phase']

In [51]:
title = "<span style='color:blue'>Budgeted</span> vs time spent with percentage \
<span style='color:red'>over budget</span> and <span style='color:green'>within budget</span>"

fig = (px.bar(df,y='phase',x=['working_hours','planned_hours'],facet_col='city',labels={'variable':'Category','planned_hours':'planned'},
        color_discrete_map={'planned_hours':'blue','working_hours':'red'},
        category_orders={'phase':['Planning','Fieldwork','Draft report','Finalization','Overall']},barmode='group')
    .update_yaxes(showgrid=False)
    .update_layout(title=title,title_x=0.5,legend_traceorder="reversed",showlegend=False)
    .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    .update_xaxes(showgrid=False)
    .for_each_xaxis(lambda xaxis: xaxis.update(title='',showticklabels=False))
    .add_annotation(y=-0.2,x=0.5,showarrow=False,
                   text="Hours", textangle=0,font_size=34,
                    xref="paper", yref="paper")
)

for d in df.iterrows():
    if d[1]['percentage']/100 >= 1:
        fig.add_annotation(y=d[1]['phase'],x=d[1]['working_hours'],text = d[1]['disp'],xshift=29,row=0,col=city_mapper[d[1]['city']],
            yshift=-12,showarrow=False)
        fig.add_annotation(y=d[1]['phase'],x=d[1]['planned_hours'],text = d[1]['planned_hours'],xshift=29,row=0,col=city_mapper[d[1]['city']],
            yshift=+12,showarrow=False)
    else:
        fig.add_annotation(y=d[1]['phase'],x=d[1]['working_hours'],text = d[1]['disp'],xshift=25,row=0,col=city_mapper[d[1]['city']],
            yshift=-12,showarrow=False)
        fig.add_annotation(y=d[1]['phase'],x=d[1]['planned_hours'],text = d[1]['planned_hours'],xshift=25,row=0,col=city_mapper[d[1]['city']],
            yshift=+12,showarrow=False)
        

for c in city_mapper:
    fig.update_traces(row=0,col=city_mapper[c],selector=dict(legendgroup = 'working_hours'),
        patch=dict(marker_color = df[df['city']==c]['colour'],showlegend = False))



In [52]:
fig

In [357]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

category_order = ['Planning','Fieldwork','Draft report','Finalization','Overall']
category_order.reverse()

f = make_subplots(rows=1,cols=len(offices),subplot_titles=list(city_mapper.keys()),shared_yaxes=True,)
for d in city_mapper:
    data = df.loc[df['city']==d].sort_values('phase_num',ascending=False)
    # data['colour'] = 'red'
    # data['colour'].mask(data['percentage']<=100,'green',inplace=True)
    
    col = city_mapper[d]
    f.add_trace(go.Bar(x=data['working_hours'],y=data['phase'],marker_color=data['colour'],orientation='h',
    showlegend= False),row=1, col=col)
    f.add_trace(go.Bar(x=data['planned_hours'],y=data['phase'],marker_color='blue',orientation='h',
    showlegend=False),row=1, col=col)
f.update_layout(barmode='group')
f.update_xaxes(showticklabels=False,showgrid=False)
f.update_yaxes(categoryorder='array', categoryarray= category_order,showgrid=False)
